# Import dependencies

In [18]:
from ucimlrepo import fetch_ucirepo, list_available_datasets
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Config Params

In [19]:
TEST_SIZE = 0.2
RANDOM_SEED = 42
K_FOLDS = 3

# Our chosen datasets
TODO
We have chosen the datasets based on the following criteria:
- datasets where we do not need to do any special preprocessing so that it is easy to do in only one pipeline
- rather small datasets to ensure we do not need high computational power

In [20]:
# Dataset that seems useful
dataset_id = {"iris": 53, 
              "heart_disease": 45,
              "wine_quality": 186,
              "breast_cancer_wisconsin": 17,
              "car_evaluation": 19, # Not good maybe remove
              "abalone": 1,
              "mushroom": 73, # a lot of categorical data
              "statlog" : 144, # german credit
              "student_performance" : 320,
              "accute_inflammation" : 284,
              "credit_approval" : 143,
              "wholesale_customers" : 292,
              "glass_identifcation" : 42,
              "ilpd" : 225,
              "hcv" : 503,
              "land_mines" : 763,
              "balance_scale" : 12
              }


# Import and preprocess datasets
For the preprocessing we will do the following steps:
1. Remove any missing values. In the article the following is written: "Given that our classifiers are not oriented to data with missing features, the missing inputs are treated as zero, which should not bias the comparison results." We therefore also decided to just remove missing values and to more focus on the full pipeline instead of single datasets. Another way could have been interpolation.
2. Encode categorical data into numerical data. This we have to do to use the classifiers later on.
3. Remove certain columns if they are highly correlated to others.
4. Split the data into a train and a test set. We will use a 80/20 split.
5. Scale the data so that we have zero mean and standard deviation of one. This is done with the Standard scaler.

In [21]:
# function for converting categorical features into numerical
def encode_categorical_features(X, encoder):
    X = encoder.fit_transform(X)
    return X

In [36]:
def import_dataset(uci_id, encoder):
    # get the dataset
    dataset = fetch_ucirepo(id=uci_id) 
    # load data into dataframe for easier preprocessing
    df = pd.concat([dataset["data"]["features"],dataset["data"]["targets"] ],axis=1)
    # remove nan values
    df.dropna(inplace=True)
    # encode categorical data
    # https://stackoverflow.com/questions/29803093/check-which-columns-in-dataframe-are-categorical
    cols = df.columns
    num_cols = df._get_numeric_data().columns
    #print(num_cols)
    categorical_cols = list(set(cols) - set(num_cols))
    #print(categorical_cols)
    df.loc[:, categorical_cols] = encode_categorical_features(df[categorical_cols], encoder)
    
    # check if encoding has worked
    # https://stackoverflow.com/questions/26924904/check-if-dataframe-column-is-categorical
    for c in df.columns:
        if df[c].dtype.name == "category":
            print(f"WARNING: Column {c} still has categorical values!")
            
    # last column is target
    return df.iloc[:,:-1], df.iloc[:,-1:]

In [38]:
ordinal_encoder = OrdinalEncoder()
X, y = import_dataset(dataset_id["breast_cancer_wisconsin"], ordinal_encoder)
#print(y)
#print(X)

In [39]:
# split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)